In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [ ]:
#for tensorflow 1.X
#tf.enable_eager_execution()

In [2]:
#install tensorflow datasets if not already installed
!pip install -q tensorflow-datasets

In [3]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised = True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteE0815T/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteE0815T/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteE0815T/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
#split into 25K samples for train and 25K samples for test
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

In [11]:
#each are iterables containing 25,000 respective sentences and labels as tensors

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

#str(s.tonumpy()) needed in Python3, as opposed to just s.numpy()

for s, l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())

for s, l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

In [12]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [16]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)

#pad sequences so that the shape is uniform
#setting truncating to 'post' means when words reach maxlen, the words at the 
#end will be dropped, instead of the beginning which is default
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)


In [17]:
#helper function to plot, reverse the word index. Input has the word as key

reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

  print(decode_review(padded[3]))
  print(training_sentences[3])

In [18]:
model = tf.keras.Sequential([
                tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(6, activation='relu'),
                tf.keras.layers.Dense(1, activation='sigmoid')             
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [24]:


num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 9.3774e-05 - accuracy: 1.0000 - val_loss: 0.8189 - val_accuracy: 0.8325
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 5.8159e-05 - accuracy: 1.0000 - val_loss: 0.8553 - val_accuracy: 0.8317
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 3.6094e-05 - accuracy: 1.0000 - val_loss: 0.8908 - val_accuracy: 0.8316
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 2.2688e-05 - accuracy: 1.0000 - val_loss: 0.9214 - val_accuracy: 0.8320
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 1.4374e-05 - accuracy: 1.0000 - val_loss: 0.9569 - val_accuracy: 0.8318
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 9.1102e-06 - accuracy: 1.0000 - val_loss: 0.9883 - val_accuracy: 0.8319
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 5.7938e-06 - accuracy: 1.0000 - val_loss: 1.0223 -

In [25]:
#embedding layer, 10k words in corpus, 16 dim array

e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [26]:
# .tsv files for tensorflow projector
# projector.tensorflow.org

import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write(('\t'.join([str(x) for x in embeddings]) + "\n"))
out_v.close()
out_m.close()

In [27]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
sentence = "I liked the movie but it wasn't very realistic. Animals can't talk or sing."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 421, 2, 18, 19, 10, 284, 53, 819, 1385, 189, 739, 40, 1940]]
